# On Candy Land

My three-year-old son was given "Candy Land" as a Christmas gift this year (thank's Aunt Marianne), and he has been obsessed with it ever since. Those of you with young children will know how this goes. It suffices to say that, given the combination of homeschooling due to the pandemic and this *wonderful* gift, I have been playing roughly at least 20+ games of Candy Land (TM) a day since Christmas.

Now, owing either to the insanity caused by such a predicament, or my genuine interest in statistics, I have begun to wonder about the statistical nature of this game.

For those unfamiliar, Candy Land is a board game consisting of a "road" of colored spaces, and the objective is to reach King Kandy's Castle first.
There are no dice. The way you move is you draw a colored card from a deck, which contains a total of 44 cards. There are four special cards which send you directly to a certain space. The remainder of the cards consist of a color (orange, green, purple, red, yellow, & blue), and a value of one or two.
For each color, there are generally four single-square cards and three double-square cards. Except for red and purple, which for some reason contain only three single-square cards.

Here is the board and the cards:

<table>
    <tr>
        <td><img src="board.jpg" alt="Candy Land Board" style="width: 300px;"/></td>
        <td><img src="cards.jpg" alt="Candy Land Board" style="width: 500px;"/></td>
    </tr>
</table>

The rules are very simple, which makes it great for 3+ year-olds (and apparently 34-year-old Ph.D. engineers ...

The cards are first shuffled.
The youngest player moves first.
Players take turns drawing a card from the deck.

* For a colored-square card, the player moves forward to the next, or second-to-next squares of that color. If that square is occupied, the player moves to the following square of that color.
* If a pink special card is drawn, the player moves forward or backwards, directly to that special location on the board.
* There are two "licorice" spots. If a player lands on this spot, they lose their next turn.
* There are two special spots, which provide shortcuts forward via either a river or a strip of candy paper. These are indicated in the photo by an arrow at the first blue square, or the third green square.
* The first player to reach King Kandy's Castle (which is the final spot that can be reached with a card of any color) wins

## Preliminaries

The first thing I noticed is that the game is purely deterministic, in the sense that once the cards are shuffled, the fates are settled.
Don't tell my son this, as playing through this inevitable fate is all the fun.

The second thing I noticed is that children can figure out very young that cheating helps.
I found my son stacking the deck to give himself the Ice Cream Cone, or even a single blue square to catch the river shortcut.
This led to some moral teaching opportunities, which was definitely unexpected.

Finally, I noticed that, even when I ensured a fair shuffle, my son would go on winning runs of 5-6 games pretty frequently.
My initial assumption was that, for a two-person game, there would be a roughyl 50% chance of each player winning.
However, it may be possible that the player with the starting draw has a higher probability of winning.

Given these questions, and the fact that one of my favorite things in the world is applying complex tools to completely trivial problems, I set about to simulate the game to study the statistical properties.

Herein lies that investigation.

## Setting up the board

Here, we first define an `Enum` to represent each type of spot on the board.
There are six colors, and four special types of spots.
We also add some behavior to check whether a given spot is special or not, and some fancy Unicode printing.

In [1]:
from enum import IntEnum, auto

class Spot(IntEnum):
    # Color spot types
    RED = auto()
    PURPLE = auto()
    YELLOW = auto()
    BLUE = auto()
    ORANGE = auto()
    GREEN = auto()
    
    # Special spot types
    PEPPERMINT = auto()
    PEANUT = auto()
    LOLLIPOP = auto()
    ICE_CREAM = auto()

    @property
    def is_special(self) -> bool:
        return self in {Spot.PEPPERMINT, Spot.PEANUT, Spot.LOLLIPOP, Spot.ICE_CREAM}
    
    def __str__(self) -> str:
        char_dict = {
            Spot.RED: "\N{LARGE RED SQUARE}",
            Spot.PURPLE: "\N{LARGE PURPLE SQUARE}",
            Spot.YELLOW: "\N{LARGE YELLOW SQUARE}",
            Spot.BLUE: "\N{LARGE BLUE SQUARE}",
            Spot.ORANGE: "\N{LARGE ORANGE SQUARE}",
            Spot.GREEN: "\N{LARGE GREEN SQUARE}",
            Spot.PEPPERMINT: "\N{CANDY}",
            Spot.PEANUT: "\N{PEANUTS}",
            Spot.LOLLIPOP: "\N{LOLLIPOP}",
            Spot.ICE_CREAM: "\N{SOFT ICE CREAM}",
        }
        if char := char_dict.get(self):
                return char
        return super().__str__()
    

The board consists of 13 cycles of repeating colors.
The four special spots are inserted manually into the board at the appropriate locations.
Once the board is defined, we won't need to update it, so we save it globally as an immutable `tuple`.

In [127]:
from typing import List

spots: List[Spot] = []
for _ in range(13):
    spots.extend([Spot.RED, Spot.PURPLE, Spot.YELLOW, Spot.BLUE, Spot.ORANGE, Spot.GREEN])

SPECIAL_SPOTS = {
    Spot.PEPPERMINT: 20,
    Spot.PEANUT: 32,
    Spot.LOLLIPOP: 49,
    Spot.ICE_CREAM: 66,
}
for spot, loc in SPECIAL_SPOTS.items():
    spots.insert(loc, spot)
    
LICORICE_SPOTS = {26, 53}
SHORTCUT_SPOTS = {3: 35, 17: 24}

BOARD = tuple(spots)
    
# " ".join(map(str, BOARD))

## Preparing the deck

The deck consists of 44 cards, each of which is either a color card or a special card.
To prevent duplicate definitions, we re-use the `Spot` enum, but add a data type to allow each card to be associated with a type of `Spot` as well as a number of squares, which will be used to represent one- and two-square cards.
The deck is then represented as a list of `Card`s, the order of which does not matter since we will create a shuffled copy every time the game is played.

In [216]:
import random
from typing import NamedTuple


class Card(NamedTuple):
    color: Spot
    num: int = 1
        
    def __str__(self) -> str:
        return " ".join(str(self.color) for _ in range(self.num))
            
    
    
DECK: List[Card] = [
    Card(Spot.PEPPERMINT),
    Card(Spot.PEANUT),
    Card(Spot.LOLLIPOP),
    Card(Spot.ICE_CREAM),
]

for color in [Spot.RED, Spot.PURPLE, Spot.YELLOW, Spot.BLUE, Spot.ORANGE, Spot.GREEN]:
    # Each color has four single-square cards, except for RED and PURPLE
    num_single = 4 if color not in {Spot.RED, Spot.PURPLE} else 3
    for _ in range(num_single):
        DECK.append(Card(color, num=1))
        
    # Each color has three two-square cards
    for _ in range(3):
        DECK.append(Card(color, num=2))


def get_shuffled_deck() -> List[Card]:
    """Return a copy of the deck, randomly shuffled."""
    deck = list(DECK)
    random.shuffle(deck)
    return deck


assert len(DECK) == 44

## Let's play the game!

Coding up the game logic should be pretty straightforward.

In [242]:
import itertools


class GameResult(NamedTuple):
    """The results of the game.
    
    Attributes:
        winner: The number of the winner, from 1-4.
        num_players: The number of players that played the game.
        num_draws: The number of card draws that occurred during the game.
    """
    winner: int
    num_players: int
    num_draws: int
    num_shortcuts_used: int
    num_turns_lost: int
    num_special_cards: int
        

def print_deck(deck):
#     for card in deck:
#         print(str(card))
    print(" | ".join(map(str, deck)))
    

def print_board(positions):
    for i, spot in enumerate(BOARD):
        for player, position in positions.items():
            if i == position:
                char = {
                    1: "1️⃣",
                    2: "2️⃣",
                    3: "3️⃣",
                    4: "4️⃣",
                }[player]
                print(char, end=" ")
                break
        else:
            print(str(spot), end=" ")
    print()
        
        
def play_game(num_players: int = 2, verbose: bool = False) -> GameResult:
    """Play a game of Candy Land, given the number of players from 2-4. Return the winner."""
    
    if not 2 <= num_players <= 4:
        raise ValueError("Number of players must be between 2-4, inclusive.")
    
    cards = get_shuffled_deck()
    
    # Mapping of player ID to current position
    positions = {i + 1: -1 for i in range(num_players)}
    player_cycle = itertools.cycle(positions.keys())
    
    # A set of players who lose their next turn if they land on a LICORICE_SPOT
    loses_next_turn = set()
    
    # Keep track of odd occurrences
    num_shortcuts_used = 0
    num_turns_lost = 0
    num_special_cards = 0
    
    for turn, card in enumerate(cards, start=1):        
        current_player = next(player_cycle)
        if current_player in loses_next_turn:
            loses_next_turn.remove(current_player)
            current_player = next(player_cycle)
        
        if card.color.is_special:
            num_special_cards += 1
            new_position = BOARD.index(card.color)
        else:
            new_position = positions[current_player]
            for step in range(card.num):
                try:
                    idx = BOARD[new_position+1:].index(card.color)
                except ValueError:
                    if verbose:
                        print(f"Player {current_player} wins!")
                    return GameResult(
                        winner=current_player, 
                        num_players=num_players, 
                        num_draws=turn,
                        num_shortcuts_used=num_shortcuts_used,
                        num_turns_lost=num_turns_lost,
                        num_special_cards=num_special_cards,
                    )
                
                new_position += idx + 1
            
            
        if new_position in LICORICE_SPOTS:
            if verbose:
                print(f"Player {current_player} loses their next turn")
            num_turns_lost += 1
            loses_next_turn.add(current_player)
        elif new_position in SHORTCUT_SPOTS:
            num_shortcuts_used += 1
            new_position = SHORTCUT_SPOTS[new_position]
        
        positions[current_player] = new_position
        
        if verbose:
            print(f"Turn: {turn}, current player: {current_player}, card: {card}")
            print()
            print_board(positions)
            print()
            
    # If we have gotten this far, we have reached the last card without a winner, which I'm not sure is even possible
    if verbose:
        print("No winner after all cards are exhausted")
        print("Deck:")
        print_deck(cards)
        print("Final Positions:")
        print_board(positions)
        
    return GameResult(
        winner=None, 
        num_players=num_players, 
        num_draws=turn,
        num_shortcuts_used=num_shortcuts_used,
        num_turns_lost=num_turns_lost,
        num_special_cards=num_special_cards,
    )

In [243]:
# Here is a random seed that leads to the game not having a winner
random.seed(891)
play_game(verbose=True)

Turn: 1, current player: 1, card: 🟧

🟥 🟪 🟨 🟦 1️⃣ 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🍬 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🥜 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🍭 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🍦 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 

Turn: 2, current player: 2, card: 🟨

🟥 🟪 2️⃣ 🟦 1️⃣ 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🍬 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🥜 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🍭 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🍦 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 

Turn: 3, current player: 1, card: 🟩

🟥 🟪 2️⃣ 🟦 🟧 1️⃣ 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🍬 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🥜 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🍭 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🍦 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 

Turn: 4, current player: 2, card: 🟪

🟥 🟪 🟨 🟦 🟧 1️⃣ 🟥 2️⃣ 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🍬 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🥜 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🍭 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🍦 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 

Turn: 5, current player: 1, card: 🟩 🟩

🟥 🟪 🟨 🟦 🟧 🟩 🟥 2️⃣ 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🍬 🟨 🟦 🟧 1️⃣ 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🥜 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🍭 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨 🟦 🟧 🟩 🟥 🟪 🟨

GameResult(winner=None, num_players=2, num_draws=44, num_shortcuts_used=1, num_turns_lost=0, num_special_cards=4)

In [256]:
import pandas
import hvplot.pandas

games = [play_game() for _ in range(1_000_000)]
df = pandas.DataFrame.from_records(games, columns=GameResult._fields)

In [257]:
num_games_without_winner = df["winner"].isna().sum() / df["winner"].count()
print(f"{num_games_without_winner * 100}% of games have no winner")

0.10030050110210433% of games have no winner


In [259]:
normalized_df = df[["winner", "num_players"]].groupby("winner").count()
normalized_df.columns = ["occurrences"]
normalized_df["pct_occurrence"] = normalized_df / normalized_df.sum() * 100
normalized_df.hvplot.bar(x="winner", y="pct_occurrence")

:Bars   [winner]   (pct_occurrence)